# CS4379 Assignment 2: NumPy, Pandas, and Data I/O

**Name:** Rudy Rutiaga  
**Date:** February 2025

This notebook covers NumPy fundamentals, pandas data manipulation, and efficient data I/O techniques.

---

## Part A: NumPy Fundamentals

In [2]:
import numpy as np
import pandas as pd
import json
from pathlib import Path

print("Libraries loaded successfully")

Libraries loaded successfully


### Problem 1: Array Creation and dtypes

In [5]:
# Create 1D array with integers 0-19
a = np.arange(20)

# Reshape into 2D array (4x5)
b = a.reshape(4, 5)

# Divide b by 3 to create float array
c = b / 3

# Print shapes and dtypes
print("Array a - Shape:", a.shape, "dtype:", a.dtype)
print("Array b - Shape:", b.shape, "dtype:", b.dtype)
print("Array c - Shape:", c.shape, "dtype:", c.dtype)

Array a - Shape: (20,) dtype: int64
Array b - Shape: (4, 5) dtype: int64
Array c - Shape: (4, 5) dtype: float64


**Explanation:** Array `c` has dtype `float64` because when we divide an integer array by a number, NumPy automatically converts the result to floating-point to preserve decimal precision. Integer division would lose information (e.g., 1/3 would become 0), so NumPy uses float64 as the default type for division operations.

### Problem 2: Indexing, Slicing, and Boolean Masks

In [9]:
# 1. Extract last column
last_col = b[:, -1]  
print("Last column:", last_col)

# 2. Extract sub-matrix (rows 1-2, columns 2-4)
sub = b[1:3, 2:5]  
print("Sub-matrix:\n", sub)

# 3. Boolean mask for divisible by 3
mask = b % 3 == 0 
divisible_by_3 = b[mask]
print("Values divisible by 3:", divisible_by_3)

# 4. Replace values > 12 with -1
b[b > 12] = -1  
print("Array b after replacement:\n", b)

Last column: [ 4  9 -1 -1]
Sub-matrix:
 [[ 7  8  9]
 [12 -1 -1]]
Values divisible by 3: [ 0  3  6  9 12]
Array b after replacement:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 -1 -1]
 [-1 -1 -1 -1 -1]]


**Explanation:** 
- Used negative indexing (`-1`) to extract the last column
- Slicing `[1:3, 2:5]` extracts rows at index 1 and 2, columns at index 2, 3, and 4 (end is exclusive)
- Boolean mask `b % 3 == 0` creates a True/False array, then `b[mask]` extracts only True positions
- Boolean indexing `b[b > 12] = -1` finds all values greater than 12 and replaces them in place without loops

### Problem 3: Vectorized Computation

In [15]:
# Set random seed for reproducibility
np.random.seed(27)  

# Create array of 100 random values from standard normal
x = np.random.normal(size=100)  

# Compute z-scores
x_mean = np.mean(x)  # calculate mean of x
x_std = np.std(x, ddof=1)   # calculate std of x
z = (x - x_mean) / x_std

# Verify z-scores
print(f"Mean of z: {z.mean()}")
print(f"Std of z: {z.std()}")

# Proportion in [-1, 1]
in_range = (z >= -1) & (z <= 1)  
proportion = in_range.sum()/len(z)  
print(f"Proportion in [-1, 1]: {proportion}")

print(f"\nRandom seed used: 27")

Mean of z: 2.886579864025407e-17
Std of z: 0.99498743710662
Proportion in [-1, 1]: 0.7

Random seed used: 27


**Explanation:** Z-scores standardize data by subtracting the mean and dividing by standard deviation. The resulting z-scores have mean ≈ 0 and std ≈ 1 by definition. The small mean value (2.89e-17) is effectively zero due to floating-point precision. About 70% of values fall within [-1, 1], which aligns with the empirical rule that approximately 68% of a normal distribution lies within one standard deviation of the mean.

### Problem 4: Aggregation and Axis Reasoning

In [18]:
# Set seed
np.random.seed(27)  

# Create 6x4 array with random integers 0-9
M = np.random.randint(2, 7, size=(6, 4))
print("Matrix M:")
print(M)
print()

# 1. Sum of each row
row_sums = M.sum(axis=1)  
print("Row sums:", row_sums)

# 2. Mean of each column  
col_means = M.mean(axis=0)  
print("Column means:", col_means)

# 3. Index of maximum value
max_idx = np.unravel_index(M.argmax(), M.shape)
print(f"Max value index (row, col): {max_idx}")

# 4. Column-wise standardization
M_standardized = (M - M.mean(axis=0)) / M.std(axis=0, ddof=1)
print("\nStandardized M:")
print(M_standardized)
print("\nVerify - Column means:", M_standardized.mean(axis=0))
print("Verify - Column stds:", M_standardized.std(axis=0, ddof=1))

Matrix M:
[[5 2 2 2]
 [2 2 3 3]
 [4 4 3 4]
 [3 6 5 5]
 [4 2 5 6]
 [5 4 6 4]]

Row sums: [11 10 15 19 17 19]
Column means: [3.83333333 3.33333333 4.         4.        ]
Max value index (row, col): (np.int64(3), np.int64(1))

Standardized M:
[[ 0.99796541 -0.81649658 -1.29099445 -1.41421356]
 [-1.56823136 -0.81649658 -0.64549722 -0.70710678]
 [ 0.14256649  0.40824829 -0.64549722  0.        ]
 [-0.71283244  1.63299316  0.64549722  0.70710678]
 [ 0.14256649 -0.81649658  0.64549722  1.41421356]
 [ 0.99796541  0.40824829  1.29099445  0.        ]]

Verify - Column means: [-1.29526020e-16 -9.25185854e-17 -3.70074342e-17  0.00000000e+00]
Verify - Column stds: [1. 1. 1. 1.]


**Explanation:** The `axis` parameter determines the direction of aggregation. `axis=1` collapses across columns (giving row-wise results), while `axis=0` collapses down rows (giving column-wise results). For standardization, we subtracted column means and divided by column standard deviations using broadcasting, which automatically applies the operation element-wise. The verification shows all columns now have mean ≈ 0 (small values due to floating-point precision) and std = 1.

### Problem 5: Linear Algebra Basics (10 points)

In [21]:
# Define matrix A and vector v
A = np.array([[2, 1, 0],
              [1, 3, 1],
              [0, 1, 2]])

v = np.array([1, 2, 3])

# 1. Compute A*v (matrix-vector product)
Av = np.dot(A, v)  
print("A*v =", Av)

# 2. Determinant of A
det_A = np.linalg.det(A)  
print(f"Determinant of A: {det_A}")

# 3. Eigenvalues of A
eigenvalues = np.linalg.eigvals(A)  
print(f"Eigenvalues of A: {eigenvalues}")

# 4. Solve Ax = v for x
x = np.linalg.solve(A, v)  
print(f"Solution x: {x}")

# Verify the solution
print(f"Verification A*x = {np.dot(A, x)}")

A*v = [ 4 10  8]
Determinant of A: 8.000000000000002
Eigenvalues of A: [4. 2. 1.]
Solution x: [0.5 0.  1.5]
Verification A*x = [1. 2. 3.]


**Explanation:** 
- Matrix-vector multiplication A·v gives [4, 10, 8]
- Determinant of A is 8 (slightly off due to floating-point precision)
- Eigenvalues are [4, 2, 1]
- Solution to Ax = v is x = [0.5, 0, 1.5], verified by computing A·x = v

**When to use `np.linalg.solve()` vs computing A⁻¹:** 
Use `np.linalg.solve(A, v)` instead of computing `inv(A) @ v` because it's more numerically stable and computationally efficient. Direct inversion can amplify rounding errors and is slower, especially for large matrices. `solve()` uses optimized algorithms (like LU decomposition) that are both faster and more accurate.

---

## Part B: Pandas

### Dataset Creation

Creating

In [22]:
records = [
    {"order_id": 101, "date": "2026-01-03", "customer": "Ava", "city": "Boston", "category": "Books", "units": 2, "unit_price": 12.50},
    {"order_id": 102, "date": "2026-01-03", "customer": "Noah", "city": "Boston", "category": "Games", "units": 1, "unit_price": 59.99},
    {"order_id": 103, "date": "2026-01-04", "customer": "Ava", "city": "Seattle", "category": "Books", "units": 3, "unit_price": 12.50},
    {"order_id": 104, "date": "2026-01-05", "customer": "Mia", "city": "Seattle", "category": "Garden", "units": 4, "unit_price": 7.25},
    {"order_id": 105, "date": "2026-01-06", "customer": "Liam", "city": "Austin", "category": "Books", "units": 1, "unit_price": 20.00},
    {"order_id": 106, "date": "2026-01-06", "customer": "Noah", "city": "Austin", "category": "Garden", "units": 2, "unit_price": 7.25},
    {"order_id": 107, "date": "2026-01-07", "customer": "Mia", "city": "Boston", "category": "Games", "units": 2, "unit_price": 59.99},
    {"order_id": 108, "date": "2026-01-08", "customer": "Ava", "city": "Austin", "category": "Books", "units": 2, "unit_price": 12.50},
]

df = pd.DataFrame(records)
print("Dataset created successfully")
print(df)

Dataset created successfully
   order_id        date customer     city category  units  unit_price
0       101  2026-01-03      Ava   Boston    Books      2       12.50
1       102  2026-01-03     Noah   Boston    Games      1       59.99
2       103  2026-01-04      Ava  Seattle    Books      3       12.50
3       104  2026-01-05      Mia  Seattle   Garden      4        7.25
4       105  2026-01-06     Liam   Austin    Books      1       20.00
5       106  2026-01-06     Noah   Austin   Garden      2        7.25
6       107  2026-01-07      Mia   Boston    Games      2       59.99
7       108  2026-01-08      Ava   Austin    Books      2       12.50


### Problem 6: Basic DataFrame Inspection

In [26]:
# 1. Show head and info
print("First 5 rows:")
print(df.head())

print("\nDataFrame info:")
df.info()

# 2. Convert date to datetime
df['date'] = pd.to_datetime(df['date'])
print("\nAfter date conversion:")
print(df.dtypes)

# 3. Add revenue column
df['revenue'] = df['units'] * df['unit_price']
print("\nDataFrame with revenue:")
print(df)

First 5 rows:
   order_id        date customer     city category  units  unit_price
0       101  2026-01-03      Ava   Boston    Books      2       12.50
1       102  2026-01-03     Noah   Boston    Games      1       59.99
2       103  2026-01-04      Ava  Seattle    Books      3       12.50
3       104  2026-01-05      Mia  Seattle   Garden      4        7.25
4       105  2026-01-06     Liam   Austin    Books      1       20.00

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    8 non-null      int64  
 1   date        8 non-null      object 
 2   customer    8 non-null      object 
 3   city        8 non-null      object 
 4   category    8 non-null      object 
 5   units       8 non-null      int64  
 6   unit_price  8 non-null      float64
dtypes: float64(1), int64(2), object(4)
memory usage: 580.0+ bytes

After date c

**Explanation:** 
- `df.head()` displays the first 5 rows to preview the data
- `df.info()` shows column names, data types, and memory usage
- Converting `date` from object (string) to datetime64 enables date-specific operations like filtering by date ranges or extracting month/year
- The `revenue` column is calculated by element-wise multiplication of `units` and `unit_price` columns

### Problem 7: Filtering and Sorting

In [34]:
# Use the boolean mask to filter df
filtered = df[(df['city'] == 'Boston') | (df['category'] == 'Garden')]

# Now sort the filtered DataFrame
sorted_df = filtered.sort_values(by=['date', 'revenue'], ascending=[True, False])

# Select columns
result = sorted_df[['order_id', 'date', 'city', 'category', 'revenue']]

print(result)

   order_id       date     city category  revenue
1       102 2026-01-03   Boston    Games    59.99
0       101 2026-01-03   Boston    Books    25.00
3       104 2026-01-05  Seattle   Garden    29.00
5       106 2026-01-06   Austin   Garden    14.50
6       107 2026-01-07   Boston    Games   119.98


**Explanation:** 
- Used the `|` (OR) operator to filter rows where city is Boston OR category is Garden
- The boolean condition creates a True/False mask that selects matching rows
- `sort_values()` with a list sorts by multiple columns: first by date ascending, then by revenue descending within each date
- Selected only the required columns using double bracket notation `df[['col1', 'col2']]`

### Problem 8: GroupBy Aggregations

In [37]:
# Group by city and aggregate multiple columns
summary = df.groupby('city').agg({
    'revenue': 'sum',  
    'units': 'sum',        
    'customer': 'nunique',    
    'unit_price': 'mean'    
})

# Rename columns for clarity
summary.columns = ['total_revenue', 'total_units', 'distinct_customers', 'avg_unit_price']

# Sort by total_revenue descending
summary = summary.sort_values('total_revenue', ascending=False)

print(summary)

         total_revenue  total_units  distinct_customers  avg_unit_price
city                                                                   
Boston          204.97            5                   3          44.160
Seattle          66.50            7                   2           9.875
Austin           59.50            5                   3          13.250


**Explanation:** 
- `groupby('city')` groups all orders by city
- `.agg()` allows applying different aggregation functions to different columns in one call
- `'nunique'` counts the number of unique customers per city (2 in Seattle, 3 in Boston and Austin)
- Boston has the highest total revenue ($204.97) due to high-value Games purchases, despite having fewer total units than Seattle

### Problem 9: Pivot Table

In [40]:
# Create pivot table
pivot = df.pivot_table(
    index='date',    
    columns='category',     
    values='revenue',      
    aggfunc='sum',      
    fill_value=0       
)

print(pivot)

category    Books   Games  Garden
date                             
2026-01-03   25.0   59.99     0.0
2026-01-04   37.5    0.00     0.0
2026-01-05    0.0    0.00    29.0
2026-01-06   20.0    0.00    14.5
2026-01-07    0.0  119.98     0.0
2026-01-08   25.0    0.00     0.0


**Explanation:** 
- `pivot_table()` reshapes data with dates as rows and categories as columns
- Each cell shows the total revenue for that date-category combination
- `fill_value=0` replaces NaN with 0 for dates when a category had no sales
- This format makes it easy to see sales patterns across categories over time

### Problem 10: Merging 

In [41]:
# Create customer tier DataFrame
customer_tier = pd.DataFrame({
    "customer": ["Ava", "Noah", "Mia", "Liam"],
    "tier": ["Gold", "Silver", "Gold", "Bronze"]
})

print("Customer tiers:")
print(customer_tier)
print()

# Merge with orders DataFrame
df_merged = df.merge(customer_tier, on='customer', how='left')
print("Merged DataFrame:")
print(df_merged)
print()

# Compute total revenue by tier
revenue_by_tier = df_merged.groupby('tier')['revenue'].sum().sort_values(ascending=False)
print("Total revenue by tier:")
print(revenue_by_tier)

Customer tiers:
  customer    tier
0      Ava    Gold
1     Noah  Silver
2      Mia    Gold
3     Liam  Bronze

Merged DataFrame:
   order_id       date customer     city category  units  unit_price  revenue  \
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00   
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99   
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50   
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00   
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00   
5       106 2026-01-06     Noah   Austin   Garden      2        7.25    14.50   
6       107 2026-01-07      Mia   Boston    Games      2       59.99   119.98   
7       108 2026-01-08      Ava   Austin    Books      2       12.50    25.00   

     tier  
0    Gold  
1  Silver  
2    Gold  
3    Gold  
4  Bronze  
5  Silver  
6    Gold  
7    Gold  

Total revenue by tier:
tier
Gol

**Explanation:** Merged the orders DataFrame with customer_tier using a left join on the 'customer' column, adding tier information to each order. Gold tier generates the most revenue, followed by Silver and Bronze.

### Problem 11: Missing Data

In [42]:
# Make a copy of df
df2 = df.copy()

# 1. Set unit_price to NaN for order_id 106
df2.loc[df2['order_id'] == 106, 'unit_price'] = np.nan
print("After setting unit_price to NaN for order 106:")
print(df2)
print()

# 2. Count missing values
print("Missing values per column:")
print(df2.isnull().sum())
print()

# 3. Replace missing unit_price with median within same category
df2['unit_price'] = df2.groupby('category')['unit_price'].transform(lambda x: x.fillna(x.median()))
print("After filling missing unit_price with category median:")
print(df2[['order_id', 'category', 'unit_price']])
print()

# 4. Recompute revenue
df2['revenue'] = df2['units'] * df2['unit_price']
print("Order 106 revenue after fix:")
print(df2[df2['order_id'] == 106][['order_id', 'units', 'unit_price', 'revenue']])

After setting unit_price to NaN for order 106:
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00
5       106 2026-01-06     Noah   Austin   Garden      2         NaN    14.50
6       107 2026-01-07      Mia   Boston    Games      2       59.99   119.98
7       108 2026-01-08      Ava   Austin    Books      2       12.50    25.00

Missing values per column:
order_id      0
date          0
customer      0
city          0
category      0
units         0
unit_price    1
revenue       0
dtype: int64

After filling missing unit_price with category median:
   order_id category  unit

**Explanation:** Created a copy to avoid modifying the original. Set order 106's unit_price to NaN, then filled it with the median unit_price of the same category (Garden). The `groupby().transform()` pattern applies the filling operation within each group while maintaining the original DataFrame shape. Revenue was successfully recomputed for order 106.

---

## Part C: Data I/O and Formats

### Problem 12: CSV I/O with Schema Control

In [44]:
# 1. Write df to CSV without index
df.to_csv('../data/orders.csv', index=False)
print("Written to orders.csv")

# 2. Read it back with proper dtypes
df_csv = pd.read_csv('../data/orders.csv', 
                      parse_dates=['date'],
                      dtype={'order_id': 'int64'})

# 3. Confirm dtypes
print("\nDataFrame dtypes after reading:")
print(df_csv.dtypes)
print("\nFirst few rows:")
print(df_csv.head())

Written to orders.csv

DataFrame dtypes after reading:
order_id               int64
date          datetime64[ns]
customer              object
city                  object
category              object
units                  int64
unit_price           float64
revenue              float64
dtype: object

First few rows:
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00


**Explanation:** Wrote the DataFrame to CSV without the index column to avoid creating an unnecessary numbered column. When reading back, explicitly parsed 'date' as datetime and ensured 'order_id' is int64 to maintain data types. This prevents common issues like dates being read as strings or IDs as floats.

### Problem 13: Efficient CSV Reading for Larger Files

In [45]:
# 1. Generate synthetic sensor data
np.random.seed(42)
n_rows = 200000

sensor_data = pd.DataFrame({
    'timestamp': pd.date_range('2026-01-01', periods=n_rows, freq='1min'),
    'sensor_id': np.random.choice(['S001', 'S002', 'S003', 'S004', 'S005'], n_rows),
    'temperature': np.random.normal(22, 5, n_rows),
    'humidity': np.random.uniform(30, 70, n_rows),
    'pressure': np.random.normal(1013, 10, n_rows),
    'voltage': np.random.uniform(3.0, 3.3, n_rows),
    'status': np.random.choice(['OK', 'WARNING', 'ERROR'], n_rows, p=[0.9, 0.08, 0.02]),
    'location': np.random.choice(['Building A', 'Building B', 'Building C'], n_rows),
    'reading_quality': np.random.uniform(0, 100, n_rows),
    'battery_pct': np.random.uniform(0, 100, n_rows)
})

sensor_data.to_csv('../data/sensor_data_large.csv', index=False)
print(f"Generated sensor_data_large.csv with {n_rows} rows")

# 2. Read only sensor_id and temperature columns
df_subset = pd.read_csv('../data/sensor_data_large.csv', usecols=['sensor_id', 'temperature'])
memory_mb = df_subset.memory_usage(deep=True).sum() / (1024**2)
print(f"\nMemory usage with usecols: {memory_mb:.2f} MB")
print(f"Shape: {df_subset.shape}")

# 3. Compute average temperature per sensor using chunking
chunk_size = 50000
sensor_sums = {}
sensor_counts = {}

for chunk in pd.read_csv('../data/sensor_data_large.csv', 
                          usecols=['sensor_id', 'temperature'],
                          chunksize=chunk_size):
    grouped = chunk.groupby('sensor_id')['temperature'].agg(['sum', 'count'])
    
    for sensor in grouped.index:
        sensor_sums[sensor] = sensor_sums.get(sensor, 0) + grouped.loc[sensor, 'sum']
        sensor_counts[sensor] = sensor_counts.get(sensor, 0) + grouped.loc[sensor, 'count']

avg_temp = pd.DataFrame({
    'sensor_id': list(sensor_sums.keys()),
    'avg_temperature': [sensor_sums[s] / sensor_counts[s] for s in sensor_sums.keys()]
}).sort_values('sensor_id')

print("\nAverage temperature per sensor:")
print(avg_temp)

Generated sensor_data_large.csv with 200000 rows

Memory usage with usecols: 11.63 MB
Shape: (200000, 2)

Average temperature per sensor:
  sensor_id  avg_temperature
0      S001        22.021851
1      S002        21.990635
2      S003        22.003920
3      S004        21.965210
4      S005        22.044571


**Explanation:** 
Generated a 200,000-row synthetic dataset. Using `usecols` loaded only 2 columns instead of 10, significantly reducing memory usage. Chunked reading processes the file in 50,000-row batches, computing partial aggregations and combining them. This is helpful even for files that fit in memory because it:
1. Reduces peak memory usage (important on memory-constrained systems)
2. Allows processing files larger than available RAM
3. Enables early termination if you find what you need in early chunks

### Problem 14: JSON Formats

In [46]:
# 1. Export as records format
df.to_json('../data/orders_records.json', orient='records', indent=2)
print("Exported orders_records.json")

# 2. Export as table format
df.to_json('../data/orders_table.json', orient='table', indent=2)
print("Exported orders_table.json")

# 3. Read both back
df_records = pd.read_json('../data/orders_records.json', orient='records')
df_table = pd.read_json('../data/orders_table.json', orient='table')

print("\nFrom records format:")
print(df_records.head())
print("\nFrom table format:")
print(df_table.head())

# Verify they match original
print("\nRecords matches original:", df_records.equals(df[df_records.columns]))
print("Table matches original:", df_table.equals(df[df_table.columns]))

Exported orders_records.json
Exported orders_table.json

From records format:
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00

From table format:
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    

**Explanation:** 
The 'records' format creates a simple JSON array of objects, making it easy to integrate with web APIs and JavaScript applications. The 'table' format includes schema metadata (column names, data types, index info), making it better for data archival and ensuring data types are preserved when reloading. Records is more portable, while table is more robust for round-trip serialization.

### Problem 15: Robust Path Handling

In [48]:
from pathlib import Path

# 1. Create data folder if it doesn't exist
data_dir = Path('../data')
data_dir.mkdir(parents=True, exist_ok=True)
print(f"Ensured {data_dir} exists")

# 2. Write df to data/orders.csv
output_path = data_dir / 'orders.csv'
df.to_csv(output_path, index=False)
print(f"Written to {output_path}")

# 3. Read it back
df_read = pd.read_csv(output_path, parse_dates=['date'])
print(f"\nRead back from {output_path}")
print(df_read.head())

Ensured ../data exists
Written to ../data/orders.csv

Read back from ../data/orders.csv
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00


**Explanation:** Using `pathlib.Path` provides cross-platform path handling without hard-coding separators like `/` or `\`. The `/` operator joins paths correctly on any OS. `mkdir(parents=True, exist_ok=True)` creates the directory if needed without raising errors if it already exists.

### Problem 16: Auto-loader Function

In [50]:
def load_orders(path):
    """
    Load orders data from CSV or JSON file.
    
    Parameters:
    path (str or Path): Path to the data file
    
    Returns:
    pd.DataFrame: Loaded orders data
    
    Raises:
    ValueError: If file extension is not .csv or .json
    """
    path = Path(path)
    
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    
    extension = path.suffix.lower()
    
    if extension == '.csv':
        return pd.read_csv(path, parse_dates=['date'])
    elif extension == '.json':
        return pd.read_json(path)
    else:
        raise ValueError(f"Unsupported file type: {extension}. Only .csv and .json are supported.")

# Test with CSV
print("Loading from CSV:")
df_csv_test = load_orders('../data/orders.csv')
print(df_csv_test.head())
print()

# Test with JSON
print("Loading from JSON (records):")
df_json_test = load_orders('../data/orders_records.json')
print(df_json_test.head())
print()

print("Function successfully loads both CSV and JSON formats!")

Loading from CSV:
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00

Loading from JSON (records):
   order_id       date customer     city category  units  unit_price  revenue
0       101 2026-01-03      Ava   Boston    Books      2       12.50    25.00
1       102 2026-01-03     Noah   Boston    Games      1       59.99    59.99
2       103 2026-01-04      Ava  Seattle    Books      3       12.50    37.50
3       104 2026-01-05      Mia  Seattle   Garden      4        7.25    29.00
4       105 2026-01-06     Liam   Austin    Books      1       20.00    20.00

Function succes